[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/QuantLet/EMQA/blob/main/EMQA_gas_seasonality/EMQA_gas_seasonality.ipynb)

# EMQA_gas_seasonality

Natural gas prices and seasonal patterns.

**Output:** `gas_prices_seasonality.pdf`

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

plt.rcParams.update({
    'figure.facecolor': 'none',
    'axes.facecolor': 'none',
    'savefig.facecolor': 'none',
    'savefig.transparent': True,
    'axes.grid': False,
    'axes.spines.top': False,
    'axes.spines.right': False,
    'font.size': 11,
    'figure.figsize': (12, 6),
})

COLORS = {
    'blue': '#1A3A6E', 'red': '#CD0000', 'green': '#2E7D32',
    'orange': '#E67E22', 'purple': '#8E44AD', 'gray': '#808080',
    'cyan': '#00BCD4', 'amber': '#B5853F'
}

def save_fig(fig, name):
    fig.savefig(name, bbox_inches='tight', transparent=True, dpi=300)
    print(f"Saved: {name}")


In [ ]:
import yfinance as yf

def fetch(ticker, start='2020-01-01', end='2025-12-31'):
    d = yf.download(ticker, start=start, end=end, progress=False)
    if isinstance(d.columns, pd.MultiIndex):
        return d['Close'].squeeze().dropna()
    return d['Close'].dropna()


In [ ]:
gas = fetch('NG=F')

fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(14, 9))
ax1.plot(gas.index, gas, color=COLORS['green'], lw=0.8)
ax1.axhline(gas.mean(), color=COLORS['red'], ls='--', lw=0.8, label=f'Mean ${gas.mean():.2f}')
ax1.set_title('Henry Hub Natural Gas Prices (2020-2025)', fontweight='bold')
ax1.set_ylabel('USD/MMBtu')
ax1.legend(loc='upper center', bbox_to_anchor=(0.5, -0.08), frameon=False)

monthly = gas.groupby(gas.index.month)
bp = ax2.boxplot([monthly.get_group(m).values for m in range(1,13)], patch_artist=True, widths=0.6)
months = ['Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sep','Oct','Nov','Dec']
ax2.set_xticklabels(months)
for patch, m in zip(bp['boxes'], range(1,13)):
    patch.set_facecolor(COLORS['red'] if m in [11,12,1,2] else COLORS['blue'])
    patch.set_alpha(0.5)
ax2.set_title('Monthly Price Distribution (Seasonal Pattern)', fontweight='bold')
ax2.set_ylabel('USD/MMBtu')

fig.tight_layout()
save_fig(fig, 'gas_prices_seasonality.pdf')
plt.show()
